In [1]:
%pip show flask

Name: Flask
Version: 2.2.3
Summary: A simple framework for building complex web applications.
Home-page: https://palletsprojects.com/p/flask
Author: Armin Ronacher
Author-email: armin.ronacher@active-4.com
License: BSD-3-Clause
Location: /Users/mton/miniconda3/envs/torch-gpu/lib/python3.9/site-packages
Requires: click, importlib-metadata, itsdangerous, Jinja2, Werkzeug
Required-by: dash, jupyter-dash
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip show flask-sqlalchemy

Name: Flask-SQLAlchemy
Version: 3.0.3
Summary: Add SQLAlchemy support to your Flask application.
Home-page: 
Author: 
Author-email: Armin Ronacher <armin.ronacher@active-4.com>
License: BSD-3-Clause
Location: /Users/mton/miniconda3/envs/torch-gpu/lib/python3.9/site-packages
Requires: Flask, SQLAlchemy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip show gunicorn

Note: you may need to restart the kernel to use updated packages.


#### -- [INIT FLASK APP] --
***

In [2]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from threading import Thread

# Init Flask app
app = Flask(__name__)

#### -- [SETUP DATABASE TABLE] --
***

In [3]:
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///visit_count.db"
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False
db = SQLAlchemy(app)

class Counter(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    count = db.Column(db.Integer, nullable=False)

# Setup the application context BEFORE creating the database tables
with app.app_context():
    db.create_all()

#### -- [DEFINE ROUTES] --
***

In [4]:
# adding a refresh counter
refresh_count = 0


# Define a route
@app.route("/")
def hello_world():
    global refresh_count
    refresh_count = refresh_count + 1
    counter = Counter.query.first()
    if counter is None:
        counter = Counter(count=0)
        db.session.add(counter)
    counter.count = counter.count + 1
    db.session.commit()
    return (
        f"Hello, World!\nVisit count : {counter.count}\nRefresh count : {refresh_count}"
    )


# Define Flask application with desired settings
def run_flask_app():
    app.run(port=5012, debug=True, use_reloader=False)


#### -- [START FLASK THREAD] --
***

In [5]:
flask_thread = Thread(target=run_flask_app)
flask_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5012
Press CTRL+C to quit
127.0.0.1 - - [15/Apr/2023 16:54:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 16:54:34] "GET / HTTP/1.1" 200 -


#### -- [EXIT FLASK THREAD] --
***

In [6]:
flask_thread.join(timeout=1)

127.0.0.1 - - [15/Apr/2023 16:54:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 16:54:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 16:54:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 16:55:17] "GET / HTTP/1.1" 200 -


In [8]:
# Check active ports
!lsof -i :5012

COMMAND     PID USER   FD   TYPE             DEVICE SIZE/OFF NODE NAME
python3.9 25362 mton   75u  IPv4 0x9b24dddc6432f2fd      0t0  TCP localhost:5012 (LISTEN)


In [9]:
!kill 25362

: 

: 